In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os
import random
import cv2
import math
import numpy as np
import scipy
from scipy import pi
from datetime import datetime
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
xs = []
ys = []

train_batch_pointer = 0
val_batch_pointer = 0

with open("SelfDrivingCar/Autopilot-TensorFlow-master/driving_dataset/data.txt") as f:
    for line in f:
        xs.append("SelfDrivingCar/Autopilot-TensorFlow-master/driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)
    
#get number of images
num_images = len(xs)
print(num_images)

train_xs = xs[:int(len(xs) * 0.8)]
train_ys = ys[:int(len(xs) * 0.8)]
val_xs = xs[-int(len(xs) * 0.2):]
val_ys = ys[-int(len(xs) * 0.2):]

print(len(train_xs))
print(len(val_xs))

num_train_images = len(train_xs)
num_val_images = len(val_xs)

45406
36324
9081


In [3]:
# def augment(imgName, angle):
#     name = 'Autopilot-TensorFlow-master/driving_dataset/' + str(imgName) + '.jpg'
#     current_image = cv2.imread(name, 1)
#     current_image = cv2.flip(current_image, 1)
#     angle = angle * -1.0
#     return current_image, angle

# for i in range(0, len(train_xs)):
#     flip_image, flip_angle = augment(i, ys[i])
#     new_image_index = i + num_images
#     cv2.imwrite("Autopilot-TensorFlow-master/driving_dataset/" + str(new_image_index) + ".jpg", flip_image)
#     train_xs.append("Autopilot-TensorFlow-master/driving_dataset/" + str(new_image_index) + ".jpg")
#     train_ys.append(flip_angle * scipy.pi/180)
#     num_images += 1

# num_train_images = len(train_xs)
# num_val_images = len(val_xs)

# print(num_train_images)
# print(num_val_images)
# print(num_images)

In [4]:
def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        index = (train_batch_pointer + i) % num_train_images
        train_image = cv2.imread(train_xs[index], 1)[-150:]
        #cv2_imshow(image)
        x_out.append(np.array(cv2.resize(train_image, (200, 66)))/255.0)
        #x_out.append(np.array(Image.open(train_xs[index][-150:]).resize((200,66)))/255.0)
        y_out.append([train_ys[index]])
    train_batch_pointer += batch_size
    return x_out, y_out

def LoadValBatch(batch_size):
    global val_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        index = (val_batch_pointer + i) % num_val_images
        val_image = cv2.imread(val_xs[index], 1)[-150:]
        x_out.append(np.array(cv2.resize(val_image, (200, 66)))/255.0)
        #x_out.append(np.array(Image.open(val_xs[index][-150:]).resize((200,66)))/255.0)
        y_out.append([val_ys[index]])
    val_batch_pointer += batch_size
    return x_out, y_out

In [5]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape = shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(previous_input, filter_input, strides):
    return tf.nn.conv2d(previous_input, filter_input, strides = [1, strides, strides, 1], padding ="VALID")

In [6]:
x_input = tf.placeholder(tf.float32, shape = [None, 66, 200, 3], name = "Plc_1")
y_true = tf.placeholder(tf.float32, name = "Plc_2")

x_image = x_input

# Model Architecture

#first convolutional layer
W_conv1 = weight_variable([5, 5, 3, 24])
b_conv1 = bias_variable([24])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

#second convolutional layer
W_conv2 = weight_variable([5, 5, 24, 36])
b_conv2 = bias_variable([36])
h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

#third convolutional layer
W_conv3 = weight_variable([5, 5, 36, 48])
b_conv3 = bias_variable([48])
h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

#fourth convolutional layer
W_conv4 = weight_variable([3, 3, 48, 64])
b_conv4 = bias_variable([64])
h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

#fifth convolutional layer
W_conv5 = weight_variable([3, 3, 64, 64])
b_conv5 = bias_variable([64])
h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

#FCL 1
W_fc1 = weight_variable([1152, 1164])
b_fc1 = bias_variable([1164])
h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#FCL 2
W_fc2 = weight_variable([1164, 400])
b_fc2 = bias_variable([400])
h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

#FCL 2
W_fc3 = weight_variable([400, 100])
b_fc3 = bias_variable([100])
h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)
h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

#FCL 3
W_fc4 = weight_variable([100, 50])
b_fc4 = bias_variable([50])
h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)
h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

#FCL 4
W_fc5 = weight_variable([50, 10])
b_fc5 = bias_variable([10])
h_fc5 = tf.nn.relu(tf.matmul(h_fc4_drop, W_fc5) + b_fc5)
h_fc5_drop = tf.nn.dropout(h_fc5, keep_prob)

#Output
W_fc6 = weight_variable([10, 1])
b_fc6 = bias_variable([1])
y = tf.multiply(tf.identity(tf.matmul(h_fc5_drop, W_fc6) + b_fc6), 2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
#Training the model
LOGDIR = './save'

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(y_true, y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
sess.run(tf.global_variables_initializer())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)

# merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

saver = tf.train.Saver()

# op to write logs to Tensorboard
logs_path = 'SelfDrivingCar/Autopilot-TensorFlow-master/logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 30
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
    for i in range(int(num_images/batch_size)):
        xs, ys = LoadTrainBatch(batch_size)
        train_step.run(feed_dict={x_input: xs, y_true: ys, keep_prob: 0.8})
        if i % 10 == 0:
            xs, ys = LoadValBatch(batch_size)
            loss_value = loss.eval(feed_dict={x_input:xs, y_true: ys,keep_prob: 1.0})
            print("Epoch: %d, Step: %d, Loss: %g" % (epoch, i, loss_value))

        # write logs at every iteration
        summary = merged_summary_op.eval(feed_dict={x_input:xs, y_true: ys, keep_prob: 1.0})
        summary_writer.add_summary(summary, epoch * num_images/batch_size + i)
        if i % batch_size == 0:
            if not os.path.exists(LOGDIR):
                os.makedirs(LOGDIR)
        checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
        filename = saver.save(sess, checkpoint_path)
    print("Model saved in file: %s" % filename)

print("Run the command line:\n" \
"--> tensorboard --logdir=./logs " \
"\nThen open http://0.0.0.0:6006/ into your web browser")

Epoch: 0, Step: 0, Loss: 7.86776
Epoch: 0, Step: 10, Loss: 7.72879
Epoch: 0, Step: 20, Loss: 7.56782
Epoch: 0, Step: 30, Loss: 7.50846
Epoch: 0, Step: 40, Loss: 7.42819
Epoch: 0, Step: 50, Loss: 7.42371
Epoch: 0, Step: 60, Loss: 7.52274
Epoch: 0, Step: 70, Loss: 8.29817
Epoch: 0, Step: 80, Loss: 8.92281
Epoch: 0, Step: 90, Loss: 7.3895
Epoch: 0, Step: 100, Loss: 7.12843
Epoch: 0, Step: 110, Loss: 7.01951
Epoch: 0, Step: 120, Loss: 6.97072
Epoch: 0, Step: 130, Loss: 6.91977
Epoch: 0, Step: 140, Loss: 6.88246
Epoch: 0, Step: 150, Loss: 6.87336
Epoch: 0, Step: 160, Loss: 6.79903
Epoch: 0, Step: 170, Loss: 6.69946
Epoch: 0, Step: 180, Loss: 6.7411
Epoch: 0, Step: 190, Loss: 6.94701
Epoch: 0, Step: 200, Loss: 6.58134
Epoch: 0, Step: 210, Loss: 6.60748
Epoch: 0, Step: 220, Loss: 6.42814
Epoch: 0, Step: 230, Loss: 6.37716
Epoch: 0, Step: 240, Loss: 6.32755
Epoch: 0, Step: 250, Loss: 6.27882
Epoch: 0, Step: 260, Loss: 6.24831
Epoch: 0, Step: 270, Loss: 11.032
Epoch: 0, Step: 280, Loss: 7.024
E

In [7]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, "save/model.ckpt")

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from save/model.ckpt


In [8]:
img = cv2.imread('SelfDrivingCar/Autopilot-TensorFlow-master/steering_wheel_image.jpg',0)
#img = Image.open('SelfDrivingCar/Autopilot-TensorFlow-master/steering_wheel_image.jpg')

rows,cols = img.shape
smoothed_angle = 0

xs = []
ys = []
#points to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0
#read data.txt

with open("SelfDrivingCar/Autopilot-TensorFlow-master/driving_dataset/data.txt") as f:
    for line in f:
        xs.append("SelfDrivingCar/Autopilot-TensorFlow-master/driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)
    
#get number of images
num_images = len(xs)
print(num_images)

i = math.ceil(num_images*0.8)
print("Starting frameofvideo:" +str(i))

while(cv2.waitKey(10) != ord('q')):
    full_image = cv2.imread(xs[i], 1)
    show_image = cv2.resize(full_image, (700, 400))
    image = np.array(cv2.resize(full_image[-150:], (200, 66)))/255.0
    #full_image = Image.open(xs[i][-150:])
    #image = np.array(full_image.resize((200,66)))/255.0
    degrees = y.eval(feed_dict={x_input: [image], keep_prob: 1.0})[0][0] * 180.0 / scipy.pi
    
    print("Steering angle: " + str(degrees) + " (pred)\t" + str(ys[i]*180.0 /scipy.pi) + "(actual)")
    cv2.imshow("frame",show_image)
    #cv2_imshow(full_image)
    
    smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    cv2.imshow("steering wheel",dst)
    #cv2_imshow(dst)
    i += 1
cv2.destroyAllWindows()

45406
Starting frameofvideo:36325
Steering angle: -4.679880840979011 (pred)	-10.79(actual)
Steering angle: 0.8584147597926175 (pred)	-10.08(actual)
Steering angle: 1.233641447131023 (pred)	-9.380000000000003(actual)
Steering angle: -14.925793827175424 (pred)	-7.559999999999999(actual)
Steering angle: -12.917408291358086 (pred)	-5.949999999999999(actual)
Steering angle: -13.988527094735948 (pred)	-3.23(actual)
Steering angle: -6.74072820603757 (pred)	-0.81(actual)
Steering angle: -20.903247539690422 (pred)	1.21(actual)
Steering angle: -19.97471747297304 (pred)	3.4300000000000006(actual)
Steering angle: -8.022826430235249 (pred)	4.44(actual)
Steering angle: -5.866792695129745 (pred)	5.340000000000001(actual)
Steering angle: -7.24663007257666 (pred)	5.75(actual)
Steering angle: -6.097992890987384 (pred)	7.16(actual)
Steering angle: -7.864837324543684 (pred)	8.27(actual)
Steering angle: -5.102048857161975 (pred)	9.48(actual)
Steering angle: -2.265858054317431 (pred)	0.0(actual)
Steering an

Steering angle: 25.76874304023276 (pred)	47.7(actual)
Steering angle: 23.2553631992623 (pred)	47.7(actual)
Steering angle: 23.392602190255367 (pred)	47.8(actual)
Steering angle: 18.05967757122133 (pred)	47.7(actual)
Steering angle: 18.586003410926047 (pred)	47.7(actual)
Steering angle: 21.362782667048883 (pred)	47.7(actual)
Steering angle: 20.67435082513037 (pred)	47.7(actual)
Steering angle: 18.60437661979338 (pred)	47.7(actual)
Steering angle: 17.76622361617361 (pred)	47.7(actual)
Steering angle: 20.681153693543703 (pred)	47.7(actual)
Steering angle: 19.44044470816901 (pred)	47.8(actual)
Steering angle: 20.234291982190204 (pred)	47.8(actual)
Steering angle: 20.054582874938 (pred)	48.0(actual)
Steering angle: 17.73097300986717 (pred)	0.0(actual)
Steering angle: 19.275763724638125 (pred)	48.5(actual)
Steering angle: 18.693592550732088 (pred)	48.71000000000001(actual)
Steering angle: 21.589517626736626 (pred)	48.91(actual)
Steering angle: 26.11334315933283 (pred)	49.11(actual)
Steering 

Steering angle: 14.274885337009671 (pred)	24.200000000000003(actual)
Steering angle: 14.705364839638895 (pred)	24.4(actual)
Steering angle: 14.778379561866332 (pred)	24.71(actual)
Steering angle: 15.894907170393731 (pred)	24.91(actual)
Steering angle: 18.458895298019403 (pred)	25.41(actual)
Steering angle: 21.212965882699237 (pred)	25.61(actual)
Steering angle: 19.150344376003765 (pred)	25.71(actual)
Steering angle: 16.711104530926498 (pred)	25.820000000000004(actual)
Steering angle: 18.206682025180218 (pred)	26.12(actual)
Steering angle: 17.486103897933045 (pred)	26.520000000000003(actual)
Steering angle: 15.254991869697115 (pred)	26.82(actual)
Steering angle: 15.148283824294005 (pred)	27.03(actual)
Steering angle: 13.604901836039366 (pred)	27.43(actual)
Steering angle: 12.862623344025414 (pred)	27.73(actual)
Steering angle: 14.981613548167353 (pred)	28.13(actual)
Steering angle: 15.514624435522224 (pred)	28.44(actual)
Steering angle: 14.382496674930515 (pred)	28.840000000000003(actua

Steering angle: 10.220602866453286 (pred)	18.05(actual)
Steering angle: 10.102759051382117 (pred)	18.05(actual)
Steering angle: 10.882347451737264 (pred)	18.05(actual)
Steering angle: 10.551334286443643 (pred)	17.95(actual)
Steering angle: 9.001029901465197 (pred)	17.75(actual)
Steering angle: 8.582162574198641 (pred)	17.04(actual)
Steering angle: 9.470560156900321 (pred)	16.03(actual)
Steering angle: 9.263578104292256 (pred)	15.230000000000002(actual)
Steering angle: 9.580941990303252 (pred)	14.32(actual)
Steering angle: 8.298034388688926 (pred)	13.92(actual)
Steering angle: 8.577331922908149 (pred)	13.82(actual)
Steering angle: 7.998353008665447 (pred)	13.82(actual)
Steering angle: 6.756035940628038 (pred)	13.610000000000001(actual)
Steering angle: 2.8739638833888295 (pred)	13.31(actual)
Steering angle: 3.855730578931408 (pred)	12.910000000000002(actual)
Steering angle: 2.398613026120384 (pred)	12.5(actual)
Steering angle: 3.7956821170695947 (pred)	12.5(actual)
Steering angle: 5.9071

Steering angle: 0.7412360343918788 (pred)	-6.76(actual)
Steering angle: -0.030199681415211378 (pred)	-7.760000000000002(actual)
Steering angle: 0.08441345417854133 (pred)	-7.969999999999999(actual)
Steering angle: 0.6499522636819509 (pred)	-7.969999999999999(actual)
Steering angle: 1.163606821815828 (pred)	-7.969999999999999(actual)
Steering angle: 2.5570623158072787 (pred)	-8.27(actual)
Steering angle: 5.110332169077908 (pred)	-8.77(actual)
Steering angle: 5.485790655961271 (pred)	-9.08(actual)
Steering angle: 1.4329980204175934 (pred)	-9.28(actual)
Steering angle: 1.463643798562971 (pred)	-9.380000000000003(actual)
Steering angle: 0.5216881412582814 (pred)	-9.380000000000003(actual)
Steering angle: 1.6243359516185687 (pred)	-9.380000000000003(actual)
Steering angle: 1.5342056291047266 (pred)	-9.18(actual)
Steering angle: 1.0208481841997452 (pred)	-8.87(actual)
Steering angle: 0.7134508248482669 (pred)	-8.67(actual)
Steering angle: -0.3799642773014556 (pred)	-8.570000000000002(actual)

...

Steering angle: 11.177183666318266 (pred)	10.08(actual)
Steering angle: 12.827041473544233 (pred)	10.08(actual)
Steering angle: 11.415979886306605 (pred)	10.08(actual)
Steering angle: 10.073574506832298 (pred)	10.08(actual)
Steering angle: 10.691453055945516 (pred)	10.08(actual)
Steering angle: 10.487326022410514 (pred)	10.18(actual)
Steering angle: 9.3549378608567 (pred)	10.18(actual)
Steering angle: 9.04611171130822 (pred)	10.29(actual)
Steering angle: 9.161949158310696 (pred)	10.389999999999999(actual)
Steering angle: 9.708521386262627 (pred)	10.49(actual)
Steering angle: 9.937496647998135 (pred)	10.18(actual)
Steering angle: 9.704204706707179 (pred)	9.68(actual)
Steering angle: 9.189743759364417 (pred)	9.68(actual)
Steering angle: 10.658176374309214 (pred)	9.68(actual)
Steering angle: 11.53907440930222 (pred)	9.68(actual)
Steering angle: 12.274190594197767 (pred)	9.68(actual)
Steering angle: 13.006319425105081 (pred)	10.08(actual)
Steering angle: 13.114607805527402 (pred)	11.290000

Steering angle: 12.597052782490625 (pred)	9.08(actual)
Steering angle: 12.163760949470648 (pred)	8.570000000000002(actual)
Steering angle: 12.775448785874895 (pred)	8.17(actual)
Steering angle: 11.805508989767016 (pred)	7.969999999999999(actual)
Steering angle: 11.805782197333818 (pred)	7.559999999999999(actual)
Steering angle: 12.21706545330072 (pred)	7.460000000000001(actual)
Steering angle: 12.14196752337643 (pred)	7.16(actual)
Steering angle: 11.924617243608283 (pred)	6.860000000000001(actual)
Steering angle: 11.980163757033413 (pred)	6.35(actual)
Steering angle: 11.486259971545097 (pred)	5.849999999999999(actual)
Steering angle: 11.539725838594311 (pred)	5.04(actual)
Steering angle: 11.710081001778184 (pred)	4.34(actual)
Steering angle: 11.44563229881457 (pred)	4.03(actual)
Steering angle: 11.540941612266574 (pred)	4.03(actual)
Steering angle: 11.260956790262117 (pred)	0.0(actual)
Steering angle: 11.31117063349276 (pred)	4.03(actual)
Steering angle: 11.310420166457703 (pred)	3.930

Steering angle: 10.812737578811078 (pred)	12.5(actual)
Steering angle: 10.5872405909104 (pred)	12.61(actual)
Steering angle: 10.448145496010383 (pred)	12.61(actual)
Steering angle: 10.031614947213617 (pred)	12.61(actual)
Steering angle: 9.76625099890122 (pred)	12.61(actual)
Steering angle: 9.571347282066675 (pred)	12.61(actual)
Steering angle: 9.239101267627868 (pred)	12.61(actual)
Steering angle: 9.240284597901573 (pred)	12.61(actual)
Steering angle: 8.542984609119447 (pred)	12.61(actual)
Steering angle: 9.716790184026573 (pred)	12.61(actual)
Steering angle: 9.899080249011398 (pred)	12.61(actual)
Steering angle: 8.928945792512174 (pred)	12.61(actual)
Steering angle: 9.205406236904903 (pred)	12.61(actual)
Steering angle: 9.420224224038266 (pred)	12.4(actual)
Steering angle: 9.029203578017853 (pred)	12.4(actual)
Steering angle: 8.44465122941242 (pred)	12.4(actual)
Steering angle: 9.550697912658435 (pred)	12.61(actual)
Steering angle: 9.270622590647477 (pred)	13.01(actual)
Steering angle

Steering angle: 3.62420552220137 (pred)	11.089999999999998(actual)
Steering angle: 1.7985322424369277 (pred)	10.49(actual)
Steering angle: -2.2897056598045307 (pred)	9.88(actual)
Steering angle: -1.5089783254052846 (pred)	9.18(actual)
Steering angle: -0.7891685944397357 (pred)	8.27(actual)
Steering angle: 1.6777155875295109 (pred)	7.659999999999998(actual)
Steering angle: -6.243685094851988 (pred)	7.26(actual)
Steering angle: -1.7938151430413876 (pred)	6.659999999999999(actual)
Steering angle: -2.4810141358147106 (pred)	6.150000000000001(actual)
Steering angle: -4.517158414192626 (pred)	5.949999999999999(actual)
Steering angle: -1.248789952936465 (pred)	5.75(actual)
Steering angle: -0.8863775542546497 (pred)	5.340000000000001(actual)
Steering angle: -2.433823505065446 (pred)	5.139999999999999(actual)
Steering angle: -3.248229648488379 (pred)	4.94(actual)
Steering angle: -3.9704444598154187 (pred)	4.64(actual)
Steering angle: -8.427199242309392 (pred)	4.24(actual)
Steering angle: -7.519

IndexError: list index out of range